<a href="https://www.kaggle.com/code/priyanagda/tdif-lr?scriptVersionId=109294457" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score
from scipy.sparse import hstack

In [2]:
sub_path = "../input/goodreads-books-reviews-290312/goodreads_sample_submission.csv"
test_path = "../input/goodreads-books-reviews-290312/goodreads_test.csv"
train_path = "../input/goodreads-books-reviews-290312/goodreads_train.csv"

In [3]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sub_df = pd.read_csv(sub_path)

In [4]:
# train_df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [5]:
# test_df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


In [6]:
train_df.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at", "started_at"], inplace=True, axis=1)
test_df.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at", "started_at"], inplace=True, axis=1)

In [7]:
# train_df.head()

,rating,review_text,n_votes,n_comments
0,5,This is a special book. It started slow for ab...,28,1
1,3,Recommended by Don Katz. Avail for free in Dec...,1,0
2,3,"A fun, fast paced science fiction thriller. I ...",22,0
3,0,Recommended reading to understand what is goin...,5,1
4,4,"I really enjoyed this book, and there is a lot...",9,1


In [8]:
# test_df.head()

,review_text,n_votes,n_comments
0,** spoiler alert ** \n This is definitely one ...,1,0
1,"** spoiler alert ** \n ""You are what you drink...",1,0
2,Roar is one of my favorite characters in Under...,0,0
3,** spoiler alert ** \n If you feel like travel...,0,0
4,3.5 stars \n I read and enjoyed the first two ...,0,0


In [9]:
train_df.loc[:,'review_text'] = train_df.review_text.apply(lambda x : str.lower(x))
test_df.loc[:,'review_text'] = test_df.review_text.apply(lambda x : str.lower(x))

In [10]:
train_text = train_df['review_text']
test_text = test_df['review_text']
all_text = pd.concat([train_text, test_text])

In [11]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

CPU times: user 5min 58s, sys: 3.88 s, total: 6min 2s
Wall time: 6min 2s


In [13]:
train_features = hstack( [train_word_features])
test_features = hstack([test_word_features])

In [14]:
# train_features

<900000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 56378834 stored elements in Compressed Sparse Row format>

In [18]:
train_target = train_df['rating']
classifier = LogisticRegression(solver='sag')
classifier.fit(train_features, train_target)

LogisticRegression(solver='sag')

In [19]:
test_pred = classifier.predict(test_features)

In [20]:
test_pred

array([4, 4, 4, ..., 5, 5, 5])

In [21]:
sub_df['rating'] = test_pred

In [22]:
sub_df.to_csv("submission.csv", index=False)